In [362]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
# #from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [363]:
# import psycopg2
# conn=psycopg2.connect(database='kenya', host='217.21.122.227', user='servir', password='Rcmrd@2022', port=5432)
# cursor=conn.cursor()
# cursor.execute('SELECT cname, season, planting, harvest, lai, gwad FROM ken_n_25_tamsat.dssat')
# rheas_raw=pd.DataFrame(cursor.fetchall())
# rheas_raw.rename(columns={0:'cname',1:'season',2:'planting',3:'harvest',4:'lai',5:'gwad'},inplace=True)

In [364]:
# Reading the data
rheas_raw=pd.read_csv('data/tamsat/ken_n_25_tamsat__newmask_2022_2023_maize.csv')
rheas_raw

,cname,season,planting,harvest,lai,gwad
0,Baringo,2021 2,2021-10-01,2022-01-16,0.69,4337.0
1,Baringo,2021 2,2021-10-26,2022-02-09,0.67,4286.0
2,Baringo,2021 2,2021-10-09,2022-01-22,0.65,4246.0
3,Baringo,2021 2,2021-10-15,2022-01-29,0.63,4328.0
4,Baringo,2021 2,2021-10-24,2022-02-07,0.67,4389.0
...,...,...,...,...,...,...
7515,Mandera,2023 1,2023-03-28,2023-07-28,1.34,3108.0
7516,Mandera,2023 1,2023-03-14,2023-07-11,1.45,3128.0
7517,Mandera,2023 1,2023-03-30,2023-07-29,1.40,3106.0
7518,Mandera,2023 1,2023-03-10,2023-07-04,1.41,3128.0


In [365]:
# remove non-agricultural counties
rheas_raw=rheas_raw.loc[(rheas_raw['cname']!='Marsabit')&
                                      (rheas_raw['cname']!='Mandera')&
                                      (rheas_raw['cname']!='Wajir')&
                                      (rheas_raw['cname']!='Lamu')&
                                      (rheas_raw['cname']!='Nairobi')&
                                      (rheas_raw['cname']!='Mombasa')&
                                      (rheas_raw['cname']!='Turkana')&
                                      (rheas_raw['cname']!='Mombasa')&
                                      (rheas_raw['cname']!='Garissa')]
                                    #   (rheas_tamsat_raw['cname']!='Isiolo')

In [366]:
# tamsat change dates to months or years and calculate yield
# rheas change dates to months or years and calculate yield
rheas_raw['planting']=pd.to_datetime(rheas_raw['planting'], format='ISO8601')
rheas_raw['harvest']=pd.to_datetime(rheas_raw['harvest'], format='ISO8601')
rheas_raw['harvest']=pd.to_datetime(rheas_raw['harvest'], format='ISO8601')
rheas_raw['year'] = rheas_raw['planting'].dt.year
rheas_raw['yield']=rheas_raw['gwad']/1000

C:\Users\kasit\AppData\Local\Temp\ipykernel_3392\3854161517.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rheas_raw['planting']=pd.to_datetime(rheas_raw['planting'], format='ISO8601')
C:\Users\kasit\AppData\Local\Temp\ipykernel_3392\3854161517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rheas_raw['harvest']=pd.to_datetime(rheas_raw['harvest'], format='ISO8601')
C:\Users\kasit\AppData\Local\Temp\ipykernel_3392\3854161517.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [367]:
# MAM and OND season OND =[10,11,12] MAM = [3,4,5]
# OND season

seasons = ['NONE', 'NONE', 'MAM', 'MAM','MAM', 'NONE', 'NONE', 'NONE', 'NONE', 'OND', 'OND', 'OND']
month_to_season=dict(zip(range(1,13), seasons))
rheas_raw['season']= rheas_raw['planting'].dt.month.map(month_to_season)
rheas_raw[rheas_raw['season']=='NONE']

C:\Users\kasit\AppData\Local\Temp\ipykernel_3392\888236314.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rheas_raw['season']= rheas_raw['planting'].dt.month.map(month_to_season)


,cname,season,planting,harvest,lai,gwad,year,yield


In [368]:
# # Extract and aggregate only for MAM and OND
# extract data for 2023 only
rheas_2023=rheas_raw[rheas_raw['year']==2023]
rheas_2022=rheas_raw[rheas_raw['year']==2022]
# rheas_2022[rheas_2022['cname']==]

In [369]:
# extract for MAM and OND
rheas_MAM_2022= rheas_2022.loc[rheas_2022['season']=='MAM']
rheas_MAM_2023= rheas_2023.loc[rheas_2023['season']=='MAM']
rheas_OND_2022= rheas_2022.loc[rheas_2022['season']=='OND']

In [370]:
# aggregate 
# rheas
rheas_MAM_2022=rheas_MAM_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_MAM_2023=rheas_MAM_2023.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_OND_2022=rheas_OND_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_MAM_2023['yield'].min()

2.021

In [371]:
# Read shapefiles and rename columns
country_shp = gpd.read_file("data/shp/gadm41_KEN_0.shp")
county_shp = gpd.read_file("data/shp/gadm41_KEN_1.shp")
maize_counties=gpd.read_file('data/shp/Kenya_maize_counties_dd.shp')


# Rename kenya_shp 
county_shp=county_shp.rename(columns={'NAME1':'cname'})

In [372]:
# merge with shapefile
# rheas
rheas_mam_2022 = pd.merge(county_shp, rheas_MAM_2022, on="cname")
rheas_mam_2023= pd.merge(county_shp, rheas_MAM_2023, on="cname")
rheas_ond_2022= pd.merge(county_shp, rheas_OND_2022, on="cname")
# tamsat_mam_2022.columns

In [373]:
# create list of min and max values
list1=[rheas_mam_2022['yield'].min(),rheas_mam_2022['yield'].max(),
       rheas_mam_2023['yield'].min(),rheas_mam_2023['yield'].max(),rheas_ond_2022['yield'].min(),rheas_ond_2022['yield'].max()]
value=(list1)



In [374]:
# convert to GeoJSONDataSource
# rheas
country_geo_source = GeoJSONDataSource(geojson=country_shp.to_json())
geo_source_rheas_mam_2022 = GeoJSONDataSource(geojson=rheas_mam_2022.to_json())
geo_source_rheas_mam_2023= GeoJSONDataSource(geojson=rheas_mam_2023.to_json())
geo_source_rheas_ond_2022= GeoJSONDataSource(geojson=rheas_ond_2022.to_json())

In [375]:
# MAM 2022
color_mapper = LinearColorMapper(palette='RdBu8', low=min(value).round(1), high=max(value).round(1))
p2 = figure(title='rheas MAM Yield (mt/ha) - 2022',
           outer_width=650, outer_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source_rheas_mam_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('County: ', '@cname'), ('Yield', '@yield')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [376]:
# MAM 2023
color_mapper = LinearColorMapper(palette="RdBu8", low=min(value).round(1), high=max(value).round(1))
p4 = figure(title='rheas MAM Yield (mt/ha) - 2023',
           outer_width=650, outer_height=600)

p4.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p4.patches('xs','ys', source=geo_source_rheas_mam_2023,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('county: ', '@cname'), ('Yield', '@yield')])
p4.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p4.add_layout(color_bar, 'right')

In [377]:
# Short Rains Season 2022
color_mapper1 = LinearColorMapper(palette='RdBu8', low=min(value).round(1), high=max(value).round(1))
p6 = figure(title='rheas OND Yield(kg/ha) - 2022',
           outer_width=650, outer_height=600)

p6.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p6.patches('xs','ys', source=geo_source_rheas_ond_2022,
          fill_color={'field': 'yield', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('County', '@cname'), ('Yield', '@yield')])
p6.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p6.add_layout(color_bar, 'right')

In [378]:
# # ploting the MAM for 2022 and 2023
# rheas_tamsat_MAM_2022= rheas_tamsat_2022.loc[rheas_tamsat_2022['season']=='MAM']
# rheas_tamsat_MAM_2023= rheas_tamsat_2023.loc[rheas_tamsat_2023['season']=='MAM']
# rheas_tamsat_MAM_2022=rheas_tamsat_MAM_2022.groupby(['year', 'cname'], as_index=False).agg({'yield': 'mean'})
# rheas_tamsat_MAM_2023=rheas_tamsat_MAM_2023.groupby(['year', 'cname'], as_index=False).agg({'yield': 'mean'})

# # starting plotting
# import matplotlib.pyplot as plt

# X=list(rheas_tamsat_MAM_2022['cname'])
# year_2023=list(rheas_tamsat_MAM_2023['yield'])
# year_2022=list(rheas_tamsat_MAM_2022['yield'])
# X_axis = np.arange(len(X))
# plt.bar(X_axis - 0.2, year_2023, 0.4, label = 'yield_2023')
# # plt.bar(X_axis + 0.2, year_2022, 0.4, label = 'yield_2022')
# plt.xticks(X_axis)
# plt.xlabel("Groups")
# plt.ylabel("Number of Students")
# plt.title("Number of Students in each group")
# plt.legend()
# plt.show()

In [379]:

p=gridplot([p2, p4, p6], ncols=2, width=650, height=600)#Tamsat only
# p=gridplot([p2, p4, p6],ncols=3, width=650, height=600)#rheas only
# p=gridplot([p1, p2, p3, p4, p5, p6],ncols=3, width=650, height=600)#rheas and tamsat


show(p)

In [380]:
from bokeh.io import save
# # specify the output file name and location
save(p, filename='../kenya.html', title='Maize Yield Maps')

C:\Users\kasit\AppData\Local\Temp\ipykernel_3392\2676417183.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(p, filename='../kenya.html', title='Maize Yield Maps')


'd:\\RONO\\rheas\\maize_crop_modeling\\crop-modeling\\kenya.html'